In [5]:
import pandas as pd
import numpy as np

data = '../new_data/books_corrected.csv'

In [7]:
import pandas as pd
import re

# Initialiser une liste pour stocker les genres
genres_list = []

# Lire le fichier CSV par morceaux
chunk_size = 10000  # Ajustez cette valeur en fonction de votre mémoire disponible
for chunk in pd.read_csv(data, delimiter=",", low_memory=False, chunksize=chunk_size):
    # Convertir la colonne 'genre_and_votes' en chaînes de caractères
    chunk['genre_and_votes'] = chunk['genre_and_votes'].astype(str)
    
    # Diviser les genres littéraires
    df_comb = chunk['genre_and_votes'].str.split(',', expand=True)
    df_comb = df_comb.fillna('None')

    # Itérer sur chaque ligne pour extraire les genres
    for index, row in df_comb.iterrows():
        for col in df_comb.columns:
            if pd.notna(row[col]):
                # Utiliser une expression régulière pour enlever les nombres
                genre = re.sub(r'\d+', '', row[col]).strip()
                genres_list.append(genre)

# Créer un DataFrame pour les genres extraits
final_genres_data = pd.DataFrame({'genre': genres_list})

# Sauvegarder les résultats finaux dans un fichier CSV
final_genres_data.to_csv("test_genres.csv", index=False)